In [293]:
!ls ./ml-100k

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [294]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os
import json
import requests
import nltk
import cPickle as pickle
import html2text
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
from scipy.stats import randint as sp_randint
from math import sqrt

In [295]:
rating = pd.read_csv('./ml-100k/u.data', sep='\t', names=['user id', 'item id' , 'rating' , 'timestamp'])
films = pd.read_csv('./ml-100k/u.item', sep='|', names=['movie id','movie title','release date','video release date',
              'IMDb URL','unknown','Action', 'Adventure','Animation','Childrens', 'Comedy','Crime','Documentary','Drama',' Fantasy',
              'Film-Noir', 'Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'] )

In [296]:
movie_title = films[['movie id','movie title']]
movie_title.columns = ['item id','movie title']

In [297]:
rating.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [298]:
hist_film = rating[rating['item id']==15].groupby('rating')['user id'].count().reset_index(name='hist_film')

In [299]:
hist_film.head()

,rating,hist_film
0,1,7
1,2,20
2,3,78
3,4,114
4,5,74


In [300]:
hist_all = rating.groupby('rating')['user id'].count().reset_index(name='hist_all')

In [301]:
hist_all

,rating,hist_all
0,1,6110
1,2,11370
2,3,27145
3,4,34174
4,5,21201


In [302]:
df = pd.concat([hist_film, hist_all], axis=1, join_axes=[hist_film.index])
del df['rating']
df.to_json('lab06.json')

In [303]:
rating.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [304]:
n_users_per_film = rating.groupby('item id')['user id'].count().reset_index(name='n')

In [305]:
sum_rating_per_film = rating.groupby('item id')['rating'].sum().reset_index(name='sum')

In [306]:
avg_rating = pd.merge(n_users_per_film, sum_rating_per_film, on='item id')

In [307]:
avg_rating = pd.merge(avg_rating, movie_title, on='item id')

In [308]:
avg_rating['avg'] = avg_rating['sum']/avg_rating['n']
avg_rating.sort_values(by=['avg','movie title'] , ascending=[False, True]).head(10)

,item id,n,sum,movie title,avg
1535,1536,1,5,Aiqing wansui (1994),5.0
1652,1653,1,5,Entertaining Angels: The Dorothy Day Story (1996),5.0
813,814,1,5,"Great Day in Harlem, A (1994)",5.0
1200,1201,1,5,Marlene Dietrich: Shadow and Light (1996),5.0
1188,1189,3,15,Prefontaine (1997),5.0
1466,1467,2,10,"Saint of Fort Washington, The (1993)",5.0
1499,1500,2,10,Santa with Muscles (1996),5.0
1598,1599,1,5,Someone Else's America (1995),5.0
1292,1293,3,15,Star Kid (1997),5.0
1121,1122,1,5,They Made Me a Criminal (1939),5.0


In [309]:
m_users_per_film  = rating[rating['rating']>3].groupby('item id')['user id'].count().reset_index(name='m')

In [310]:
avg_rating = pd.merge(m_users_per_film, avg_rating, on='item id')
avg_rating.head()

,item id,m,n,sum,movie title,avg
0,1,321,452,1753,Toy Story (1995),3.878319
1,2,51,131,420,GoldenEye (1995),3.206107
2,3,34,90,273,Four Rooms (1995),3.033333
3,4,122,209,742,Get Shorty (1995),3.550239
4,5,39,86,284,Copycat (1995),3.302326


In [311]:
avg_rating['proportion'] = avg_rating['m']/avg_rating['n']
avg_rating.head()

,item id,m,n,sum,movie title,avg,proportion
0,1,321,452,1753,Toy Story (1995),3.878319,0.710177
1,2,51,131,420,GoldenEye (1995),3.206107,0.389313
2,3,34,90,273,Four Rooms (1995),3.033333,0.377778
3,4,122,209,742,Get Shorty (1995),3.550239,0.583732
4,5,39,86,284,Copycat (1995),3.302326,0.453488


In [347]:
float(rating['rating'].sum()) / len(rating)

3.52986

In [346]:
k=20
avg_rating['imp_avg'] = (avg_rating['sum'] + k*mu) / (avg_rating['n'] + k)
avg_rating.sort_values(by=['imp_avg','movie title'], ascending=[False, True]).head(10)['item id']

314    318
474    483
63      64
404    408
49      50
166    169
11      12
590    603
97      98
124    127
Name: item id, dtype: int64

In [314]:
#avg_rating['proportion']+((1*z*z)/(2*avg_rating['n']))+z*sqrt(1/avg_rating['n']*avg_rating['proportion']*(1-avg_rating['proportion'])+1*z*z/(4*avg_rating['n']*avg_rating['n']))
def confidence(row):
    n = row['n']
    phat = row['proportion'] 
    z = 3.291
    
    if n == 0:
        return 0
    return ((phat + z*z/(2*n) - z * sqrt((phat*(1-phat)+z*z/(4*n))/n))/(1+z*z/n))

avg_rating['confidence_min'] = avg_rating.apply(confidence, axis=1)
avg_rating.head()

,item id,m,n,sum,movie title,avg,proportion,imp_avg,confidence_min
0,1,321,452,1753,Toy Story (1995),3.878319,0.710177,3.841102,0.635683
1,2,51,131,420,GoldenEye (1995),3.206107,0.389313,3.178808,0.262759
2,3,34,90,273,Four Rooms (1995),3.033333,0.377778,3.027273,0.231465
3,4,122,209,742,Get Shorty (1995),3.550239,0.583732,3.502183,0.470114
4,5,39,86,284,Copycat (1995),3.302326,0.453488,3.245283,0.292113


In [315]:
def confidence(row):
    n = row['n']
    phat = row['proportion'] 
    z = 3.291
    
    if n == 0:
        return 0
    return ((phat + z*z/(2*n) + z * sqrt((phat*(1-phat)+z*z/(4*n))/n))/(1+z*z/n))

avg_rating['confidence_max'] = avg_rating.apply(confidence, axis=1)
avg_rating.head()

,item id,m,n,sum,movie title,avg,proportion,imp_avg,confidence_min,confidence_max
0,1,321,452,1753,Toy Story (1995),3.878319,0.710177,3.841102,0.635683,0.774834
1,2,51,131,420,GoldenEye (1995),3.206107,0.389313,3.178808,0.262759,0.532772
2,3,34,90,273,Four Rooms (1995),3.033333,0.377778,3.027273,0.231465,0.550347
3,4,122,209,742,Get Shorty (1995),3.550239,0.583732,3.502183,0.470114,0.689099
4,5,39,86,284,Copycat (1995),3.302326,0.453488,3.245283,0.292113,0.625268


In [316]:
result = pd.DataFrame()
result['top10_rates'] = avg_rating.sort_values(by='n', ascending=[False]).reset_index().head(10)['item id']
result['top10_average'] = avg_rating.sort_values(by=['avg','movie title'] , ascending=[False, True]).reset_index().head(10)['item id']
result['top10_rating'] = avg_rating.sort_values(by=['imp_avg','movie title'] , ascending=[False, True]).reset_index().head(10)['item id']
result['top10_lower'] = avg_rating.sort_values(by='confidence_min', ascending=[False]).reset_index().head(10)['item id']

In [317]:
top10_rates = avg_rating.sort_values(by='n', ascending=[False]).reset_index().head(10)['item id'].tolist()
top10_average = avg_rating.sort_values(by=['avg','movie title'] , ascending=[False, True]).reset_index().head(10)['item id'].tolist()
top10_rating = avg_rating.sort_values(by=['imp_avg','movie title'] , ascending=[False, True]).reset_index().head(10)['item id'].tolist()
top10_lower = avg_rating.sort_values(by='confidence_min', ascending=[False]).reset_index().head(10)['item id'].tolist()

In [318]:
result

,top10_rates,top10_average,top10_rating,top10_lower
0,50,1536,318,64
1,258,1653,64,98
2,100,814,483,318
3,181,1201,50,479
4,294,1189,12,50
5,286,1467,603,483
6,288,1500,408,603
7,1,1599,169,427
8,300,1293,98,357
9,121,1122,127,12


In [319]:
result.to_json('lab06s.json')